In [1]:
# coding: utf-8
"""
DM計算用
"""
from library.filer import Filer
from library.mixture_of_unigram_model import MUM
from library.dnp import Evaluation
import glob
import numpy as np
from sklearn.metrics import adjusted_mutual_info_score, adjusted_rand_score, completeness_score, homogeneity_completeness_v_measure, homogeneity_score, normalized_mutual_info_score

In [3]:
list_filepath = glob.glob('files/rakuten_corpus/rakuten_corpus_master/preprocessedfile/type2/forUM/*.txt')
list_filepath.sort()
list_testfile = Filer.readdump('./files/rakuten_corpus/rakuten_corpus_master/testfile/list_sepword_label.dump')
list_testword = [row[0] for row in list_testfile]
list_label = [row[1] for row in list_testfile]

outputpath = './files/result/rakuten/experiment1/DM.txt'
removepath = 'files/rakuten_corpus/rakuten_corpus_master/preprocessedfile/type2/forUM/'

In [ ]:
for path in list_filepath:
    print path
    path_rev = path.replace(removepath, '')
    Filer.writetxt([path_rev], outputpath)
    for iteration in range(0,10):
        K = 6
        alpha = 0.01    # 初期ハイパーパラメータalpha
        beta = 0.01     # 初期ハイパーパラメータbeta
        N = 1000        # 最大イテレーション回数
        converge = 0.01 # イテレーション10回ごとに対数尤度を計算し，その差分(converge)が小さければ学習を終了する

        mum = MUM(path)
        mum.set_param(alpha, beta, K, N, converge)
        likelihood = mum.learn()
        Filer.writetxt([str(K) + ' ' + str(likelihood)], outputpath)
    
        list_predict = []
        for row in list_testword:
            try:
                list_prob = mum.infer(row)
                list_predict.append(np.argmax(list_prob))
            except KeyError:
                list_predict.append(None)

        list_predict_rev = []
        list_label_rev = []
        for predict, label in zip(list_predict, list_label):
            if predict != None:
                list_predict_rev.append(predict)
                list_label_rev.append(label)

        eva = Evaluation()
        dict_result = eva.cal_f_measure(list_predict_rev, list_label_rev)
        
        # NMI
        NMI = normalized_mutual_info_score(list_label_rev, list_predict_rev)
        # vm
        H, C, VM = homogeneity_completeness_v_measure(list_label_rev, list_predict_rev)
        # ARI
        ARI = adjusted_rand_score(list_label_rev, list_predict_rev)
        # AMI
        AMI = adjusted_mutual_info_score(list_label_rev, list_predict_rev)

        Filer.writetxt([' '.join([str(dict_result['purity']),str(dict_result['invpurity']),str(dict_result['fvalue']),
                                  str(NMI), str(H), str(C), str(VM), str(ARI), str(AMI), str(K)])],
                       outputpath)

In [11]:
# 正解データの読み込み
list_sep = Filer.readdump('./files/corpus/rakuten/all_sep.dump')
list_label = [row[1] for row in list_sep]
list_testword = [row[2] for row in list_sep]

In [12]:
path = './files/corpus/rakuten/rakuten_corpus_for_UM.txt'
for K in range(20,21):
    for num in range(10):
        alpha = 0.01    # 初期ハイパーパラメータalpha
        beta = 0.01     # 初期ハイパーパラメータbeta
        N = 1000        # 最大イテレーション回数
        converge = 0.01 # イテレーション10回ごとに対数尤度を計算し，その差分(converge)が小さければ学習を終了する
        mum = MUM(path)
        mum.set_param(alpha, beta, K, N, converge)
        perplexity = mum.learn()

        list_predict = []
        for row in list_testword:
            try:
                list_prob = mum.infer(row)
                list_predict.append(np.argmax(list_prob))
            except KeyError:
                list_predict.append(None)

        list_predict_rev = []
        list_label_rev = []
        for predict, label in zip(list_predict, list_label):
            if predict != None:
                list_predict_rev.append(predict)
                list_label_rev.append(label)

        eva = Evaluation()
        dict_score = eva.cal_f_measure(list_predict_rev, list_label_rev)
        list_tmp = [str(dict_score['K']), str(dict_score['fvalue']),
                    str(dict_score['purity']), str(dict_score['invpurity']),
                    str(perplexity)]
        Filer.writetxt([' '.join(list_tmp)], './files/DM_result.txt')

In [13]:
path = './files/corpus/rakuten/rakuten_corpus_for_UM.txt'
for K in range(3,50):
    list_ave = []
    for i in range(3):
        alpha = 0.01    # 初期ハイパーパラメータalpha
        beta = 0.01     # 初期ハイパーパラメータbeta
        N = 100        # 最大イテレーション回数
        converge = 0.01 # イテレーション10回ごとに対数尤度を計算し，その差分(converge)が小さければ学習を終了する
        mum = MUM(path)
        mum.set_param(alpha, beta, K, N, converge)
        perplexity = mum.learn()
        list_ave.append(mum.cal_ave_dis())
    print K, np.average(list_ave)

3 0.289529302683
4 0.280054719777
5 0.250646636355
6 0.252925176394
7 0.236021646034
8 0.225934602132
9 0.223262665051
10 0.226311380158
11 0.211979940481
12 0.20656644137
13 0.196238463333
14 0.193479058187
15 0.19266294799
16 0.190002568845
17 0.181602128912
18 0.17755596978
19 0.177351578156
20 0.179067433462
21 0.172415925592
22 0.170892005706
23 0.166299941842
24 0.167301317881
25 0.159761515737
26 0.163701473668
27 0.158234262148
28 0.159163700819
29 0.1496265994
30 0.15526870851
31 0.154376546485
32 0.154419838106
33 0.150796828087
34 0.15287303829
35 0.146556162093
36 0.146787050876
37 0.152776511368
38 0.163528232455
39 0.146984279268
40 0.151664862909
41 0.149895447878
42 0.156443053496
43 0.164322704336
44 0.17452946419
45 0.1704216875
46 0.175201739231
47 0.146072676382
48 0.172166149345
49 0.161476386236
